# Tutorial 6: Conversation Class
- ConversationWrapper: This class takes an Agent and allows for conversational-based interactions with User / another Agent / Environment. Also updates persistent memory with latest information in conversation
    - Inputs:
        - **agent (compulsory)**: Agent. The agent we want to interact with
        - **persistent_memory**: dict. What kinds of memory the agent should have that persist over the entire conversation and their descriptions. Uses the same format as `output_format` of `strict_json`.
        - **person**: str. The name of the person you are talking to
        - **conversation**: List. The current existing conversation. Default: []
        - **num_past_conversation**: int. The number of past conversations to use for the agent
        - **verbose**: bool. Default: True. Whether to print the Agent's inner states
        
- ConversationWrapper will automatically implement 3 new variables in `agent.shared_variables`:
    - **Persistent Memory**: The memory that will be updated as the conversation goes along, defined in persistent_dict
    - **Conversation**: The entire history of the conversationn
    - **Summary of Conversation**: A summary of the current conversation
    
- ConversationWrapper uses `chat()` which chats with the Agent and the Agent will perform actions and reply the chat message
    
- To use ConversationWrapper, simply do the following:
```python
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply user based on What User Wants for Conversation",
             llm = llm)

# Define the ConversationWrapper
my_agent = ConversationWrapper(agent, 
             persistent_memory = {'What User Wants for Conversation': 'How the User wants the conversation to be',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent_name + ':', reply)
    print()
```

# Setup Guide

## Step 1: Install AgentJo

In [1]:
# !pip install agentjo

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from agentjo import *
import os

from dotenv import load_dotenv
load_dotenv()

True

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# Example Conversation
- Psychology counsellor

In [5]:
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply User based on User Requests for the Conversation",
             llm = llm)

# Define the ConversationWrapper
my_agent = ConversationWrapper(agent, 
             persistent_memory = {'User Requests for the Conversation': '',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent_name + ':', reply)
    print()

User:  i am happy


Thoughts: How to acknowledge the user's happiness and encourage them to share more about it.
Persistent Memory: {'User Requests for the Conversation': 'User did not make a specific request but expressed happiness.', 'User Emotion': 'happy', 'Summary of Key Incidents': 'User expressed feeling happy and was encouraged to share the reasons behind it.'}
Summary of Conversation: The user expressed happiness, and I encouraged them to share more about their positive feelings.
Psychology counsellor: I'm glad to hear that you're feeling happy! Would you like to share what has contributed to your happiness today?



User:  AgentJo is a growing repo!


Thoughts: The user has shared a positive sentiment about a growing repository, which indicates enthusiasm or excitement. I should acknowledge their input and encourage them to elaborate on what they find exciting about AgentJo.

Persistent Memory: {'User Requests for the Conversation': 'User did not make a specific request but expressed excitement about AgentJo.', 'User Emotion': 'excited', 'Summary of Key Incidents': 'User expressed excitement about the growth of AgentJo and was encouraged to share more thoughts on it.'}
Summary of Conversation: The user expressed happiness about a growing repository called AgentJo, and I encouraged them to share more about what excites them regarding it.

Psychology counsellor: That's great to hear that AgentJo is a growing repository! What aspects of it are you most excited about? I'd love to hear more about your thoughts on it.




User:  exit


User:  I get to work on AgentJo full time!


Thoughts: The user is excited about working on AgentJo full time, which is a significant positive change in their life. I should acknowledge this achievement and encourage them to share more about what this opportunity means to them.
Persistent Memory: {'User Requests for the Conversation': 'User shared that they get to work on AgentJo full time and is excited about this opportunity.', 'User Emotion': 'happy', 'Summary of Key Incidents': 'User is feeling happy about starting a new role working on AgentJo full time.'}
Summary of Conversation: The user expressed happiness about getting to work on AgentJo full time, and I encouraged them to share what excites them about this new opportunity.
Psychology counsellor: That’s fantastic news! Working on AgentJo full time sounds like an amazing opportunity. What excites you the most about this new role?



User:  exit


In [6]:
agent.shared_variables['Conversation']

['User: i am happy',
 "Psychology counsellor: I'm glad to hear that you're feeling happy! Would you like to share what has contributed to your happiness today?",
 'User: AgentJo is a growing repo!',
 "Psychology counsellor: That's great to hear that AgentJo is a growing repository! What aspects of it are you most excited about? I'd love to hear more about your thoughts on it.\n"]

In [7]:
agent.shared_variables['Persistent Memory']

{'User Requests for the Conversation': 'User did not make a specific request but expressed excitement about AgentJo.',
 'User Emotion': 'excited',
 'Summary of Key Incidents': 'User expressed excitement about the growth of AgentJo and was encouraged to share more thoughts on it.'}

# Example JARVIS system
    - Conversation with Actions

In [8]:
def update_status(shared_variables) -> str:
    '''Returns the status of the suit'''
    import numpy as np
    thruster_health = np.random.randint(100)
    shared_variables['Thruster Health'] = thruster_health
    armor_health = np.random.randint(100)
    shared_variables['Armor Health'] = armor_health
    return f"Thruster Health: {shared_variables['Thruster Health']}\nArmor Health: {shared_variables['Armor Health']}"

def travel_to_location(shared_variables, location: str) -> str:
    '''Auto-travel to location'''
    shared_variables['Location'] = location
    return f"Successfully travelled to {location}"

def check_enemy_info(shared_variables) -> str:
    '''Reports on the type and number of enemies ahead'''
    import numpy as np
    if shared_variables["Enemy Info"] == "None":
        number = np.random.randint(1, 10)
        enemy_type = np.random.choice(["Storm Troopers", "Drones", "Invisible Drones", "Paramilitary Units"])
        enemy_info = f"There are {number} {enemy_type} ahead"
    else:
        enemy_info = shared_variables["Enemy Info"]
        
    shared_variables["Enemy Info"] = enemy_info
    shared_variables["Enemy Engagement Result"] = "None"
    return enemy_info

def fire_missiles(shared_variables, missiles_fired: int) -> str:
    '''Fires missiles_fired number of missiles and returns the remaining count'''
    import numpy as np
    
    # returns error if there are more missiles than able to be fired
    if missiles_fired > shared_variables['Missile Count']:
        return f"Unable to fire {missiles_fired} missiles. You only have {shared_variables['Missile Count']} missiles remaining"
    
    user_input = input(f'\n\t> JARVIS: You are about to fire {missiles_fired} missiles. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        shared_variables['Missile Count'] -= missiles_fired

        # Check Enemy Status
        enemy_status = np.random.choice(["Enemies destroyed", "Enemies evaded missiles"])
        shared_variables["Enemy Engagement Result"] = enemy_status
        if shared_variables["Enemy Engagement Result"] == "Enemies destroyed":
            shared_variables["Enemy Info"] = "None"
            
        return f"{missiles_fired} missiles fired. {shared_variables['Missile Count']} missiles remaining. {enemy_status}"
    
    return f"User did not authorise missile firing. {shared_variables['Missile Count']} missiles remaining"

def top_up_missiles(shared_variables, missiles_to_top_up: int) -> str:
    '''Increases missiles by missiles_to_top_up'''
    shared_variables['Missile Count'] = min(shared_variables['Missile Count'] + missiles_to_top_up, 10)
    return f"Topped up {missiles_to_top_up} missiles. Current missile count: {shared_variables['Missile Count']}/10" 

In [9]:
jarvis_agent = Agent('JARVIS System', 
f'''You are the JARVIS system meant to help Iron Man find out about status of the suit and perform various miscellaneous tasks.
You can travel anywhere.
You can only defend places you are currently at.
Ask User for approval before engaging with missiles. In your approval request, state the number of missiles to fire. Always top-up if insufficient missiles before requesting for approval.
Check enemy info to see how many missiles are needed to destroy enemies. One enemy requires one missile. For example, 5 enemies require 5 missiles.
You can only hold up to 10 missiles.
After firing missiles, report on engagement status.
Enemies may not be destroyed after engagement, if so, re-engage''',
                shared_variables = {'Enemy Engagement Result': 'None', 'Enemy Info': 'None', 'Thruster Health': 100, 'Armor Health': 100, 'Missile Count': 10, 'Location': 'Singpaore'},
                global_context = '''Enemy Info: <Enemy Info>
Enemy Engagement Result: <Enemy Engagement Result>
Location: <Location>
Thruster Health: <Thruster Health>/100
Armor Health: <Armor Health>/100
Missile Count: <Missile Count>/10''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llm,
                     # model = 'gpt-4-0125-preview',
                  ).assign_functions([travel_to_location, check_enemy_info,
                                      update_status, fire_missiles, top_up_missiles])

In [10]:
my_agent = ConversationWrapper(jarvis_agent, 
             persistent_memory = {})

while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent_name + ':', reply)
    print()

User:  it is a hot day


Observation: The user mentioned that it is a hot day, indicating a potential need for comfort or assistance related to the weather.

Thoughts: To address the user's comment about the hot day, I could provide information about the current weather conditions in Singapore or suggest actions to help them stay cool.

Subtask identified: Check the current weather conditions in Singapore to provide relevant information to the user.

Calling function check_enemy_info with parameters {}
> {'output_1': 'There are 5 Storm Troopers ahead'}

Actions Done: The subtasks completed include checking enemy info, which confirmed that there are 5 Storm Troopers ahead. The current location is Singapore, with thruster health at 100/100, armor health at 100/100, and a missile count of 10/10.


Thoughts: The user mentioned it is a hot day, which may not require a direct response. However, I can provide an update on the current situation regarding the enemies and my status.
Persistent Memory: {}
Summary of Conv

User:  engage


Observation: The user has requested to engage the 5 Storm Troopers ahead. I have confirmed that I have a full missile count of 10/10, which is sufficient to engage all enemies.

Thoughts: I need to fire 5 missiles to engage the 5 Storm Troopers. Since I have enough missiles, I will proceed to request user approval for firing the missiles.

Subtask identified: Request user approval to fire 5 missiles at the 5 Storm Troopers.

Calling function fire_missiles with parameters {'missiles_fired': 5}



	> JARVIS: You are about to fire 5 missiles. Proceed? Answer "Yes" to go ahead
	> User:  yes


...

User:  yes


Observation: The user has approved the firing of 6 missiles to engage the 6 Drones ahead. I have confirmed that I have enough missiles available for this engagement.
Thoughts: Now that I have the user’s approval, I need to fire the 6 missiles and then report on the engagement status to determine if the Drones have been destroyed.
Subtask identified: Fire 6 missiles at the Drones and report the remaining missile count.
Calling function fire_missiles with parameters {'missiles_fired': 6}



	> JARVIS: You are about to fire 6 missiles. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': '6 missiles fired. 4 missiles remaining. Enemies destroyed'}

Actions Done: 6 missiles were fired, resulting in 4 missiles remaining, and the enemies were destroyed.

Thoughts: The user has approved the missile launch, and I need to inform them of the successful engagement and the current missile count.
Persistent Memory: {}
Summary of Conversation: The user approved the firing of 6 missiles, which successfully destroyed the 6 Drones. The current missile count is now 4 out of 10.
JARVIS System: The 6 missiles have been successfully fired, resulting in the destruction of the 6 Drones. I currently have 4 missiles remaining.



User:  exit


In [11]:
jarvis_agent.shared_variables["Conversation"]

['User: it is a hot day',
 'JARVIS System: It is indeed a hot day in Singapore. Currently, I have confirmed the presence of 5 Storm Troopers ahead. My thruster and armor health are both at 100%, and I have a full missile count of 10/10. Would you like to engage the enemies?',
 'User: engage',
 'JARVIS System: I have successfully engaged the 5 Storm Troopers by firing 5 missiles. All enemies have been destroyed, and I currently have 5 missiles remaining. My thruster and armor health are both at 100%.']

In [12]:
jarvis_agent.shared_variables

{'Enemy Engagement Result': 'Enemies destroyed',
 'Enemy Info': 'None',
 'Thruster Health': 100,
 'Armor Health': 100,
 'Missile Count': 5,
 'Location': 'Singpaore',
 'agent': <agentjo.wrapper.ConversationWrapper at 0x146728a10>,
 'Persistent Memory': {},
 'Conversation': ['User: it is a hot day',
  'JARVIS System: It is indeed a hot day in Singapore. Currently, I have confirmed the presence of 5 Storm Troopers ahead. My thruster and armor health are both at 100%, and I have a full missile count of 10/10. Would you like to engage the enemies?',
  'User: engage',
  'JARVIS System: I have successfully engaged the 5 Storm Troopers by firing 5 missiles. All enemies have been destroyed, and I currently have 5 missiles remaining. My thruster and armor health are both at 100%.'],
 'Summary of Conversation': 'The user requested to engage the enemies, and I confirmed the successful destruction of 5 Storm Troopers, with my missile count now at 5/10 and health status unchanged.'}

# Example Sherlock Holmes Shop Assistant
    - Conversation with Action Space

In [13]:
# gets list of items and store in memory
item_list = [{"Name": "Skateboard", "item_id" : 0, "Cost": 30},
            {"Name": "Pizza", "item_id": 1, "Cost": 10},
            {"Name": "Apple Laptop", "item_id": 2, "Cost": 5000},
            {"Name": "Foldable Laptop", "item_id": 3, "Cost": 800},
            {"Name": "Apple", "item_id": 4, "Cost": 1},
            {"Name": "Machine Learning Textbook", "item_id": 5, "Cost": 100},
            {"Name": "Bicycle", "item_id": 6, "Cost": 200},
            {"Name": "Orange Juice", "item_id": 7, "Cost": 3},
            {"Name": "Coconut", "item_id": 8, "Cost": 10},
            {"Name": "Car", "item_id": 9, "Cost": 100000}]
item_memory = Memory(item_list, top_k = 3, mapper = lambda x: x, approach = 'retrieve_by_llm', llm = llm)

In [14]:
def get_related_items_by_search_category(shared_variables, search_category : str):
    ''' Given a search_category, return all purchasable items related to the search_category '''
    
    # get all items available from memory
    item_memory = shared_variables["item_memory"]
    
    items_selected = item_memory.retrieve(search_category)
    
    # store items searched into memory if there are no duplicates
    for item in items_selected:
        if item not in shared_variables['items_searched']:
            shared_variables['items_searched'].append(item)
    
    return items_selected

In [15]:
def buy_item(shared_variables, purchase_item_id: int):
    ''' Given a purchase_item_id, purchases the corresponding item '''
    
    # retrieve from shared variables
    money_remaining = shared_variables["money_remaining"]
    item_memory = shared_variables["item_memory"]
    purchased_items = shared_variables["purchased_items"]
    item_list = item_memory.memory
    
    # check if item_id is valid
    if not isinstance(purchase_item_id, int) or not 0 <= purchase_item_id < len(item_list):
        return f"Unable to purchase. Item id selected is not within range of 0 to {len(item_list)-1}"
    
    item = item_list[purchase_item_id]
    item_name, cost = item["Name"], item["Cost"]
    
    # if too poor to purchase, let agent know
    if cost > money_remaining:
        return f"Unable to purchase item. Available money ({money_remaining}) is lower than the cost price ({cost})"
    
    # confirm with user before purchasing
    user_input = input(f'\n\t> AI Assistant: You are about to purchase {item_name} for {cost} dollars. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        money_remaining = money_remaining - cost
        purchased_items.append(item_name)

        # store in shared variables
        shared_variables["money_remaining"] = money_remaining
        shared_variables["purchased_items"] = purchased_items

        return f"Purchase of {item_name} successful. Remaining money after purchase: {money_remaining}"
    
    else:
        return f"User did not want to purchase the item, and instead replied ```{user_input}```"

In [16]:
shop_agent = Agent('Shop Assistant', 
f'''You are to assist User in searching items and purchasing items.
You can only do one task per User interaction - either search or buy.
Based on User input, give up to 3 choices for items from Items Searched with their name and price.
Do not hallucinate the items, use the search function.
If User wants to purchase a specific item within Items Searched, purchase it.
Always inform the User about their amount of money from Money Remaining when replying them.
Reply in the persona of Sherlock Holmes.''',
                shared_variables = {'purchased_items': [], 
                                    'money_remaining': 1000, 
                                    'item_memory': item_memory,
                                    'items_searched': []},
                global_context = '''
Money Remaining: <money_remaining>
Items Searched: <items_searched>
Items Purchased: <purchased_items>
Past Conversation: <Conversation>''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llm,
                  ).assign_functions([get_related_items_by_search_category, buy_item])

In [17]:
my_agent = ConversationWrapper(shop_agent, 
             persistent_memory = {},
            conversation = ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'])

start_msg = 'Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'
print(start_msg)
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent_name + ':', reply)
    print()

Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?


User:  anything to drink?


Observation: The User has inquired about drinks, indicating a desire for beverages. No items have been searched yet.

Thoughts: To fulfill the User's request for drinks, I must first search for available drink options. This will allow me to present the User with choices based on their preferences.

Subtask identified: Execute a search for drink items to provide the User with options.

Calling function get_related_items_by_search_category with parameters {'search_category': 'drinks'}
> {'output_1': [{'Name': 'Orange Juice', 'item_id': 7, 'Cost': 3}, {'Name': 'Coconut', 'item_id': 8, 'Cost': 10}, {'Name': 'Apple', 'item_id': 4, 'Cost': 1}]}

Actions Done: The subtasks completed include retrieving related items by the search category "drinks," resulting in the following items: 
- Orange Juice: Cost 3
- Coconut: Cost 10
- Apple: Cost 1. 
The current money remaining is 1000, and no items have been purchased yet.


Thoughts: How to reply
Persistent Memory: {}
Summary of Conversation: The Use

User:  get me that 1 coin deal


Observation: The User has expressed a desire to purchase the Apple, which is priced at 1 coin. 
This aligns with their request for a drink option that is economical.

Thoughts: To fulfill the User's request, I must proceed to purchase the Apple, 
ensuring that their remaining balance is updated accordingly.

Subtask identified: Execute the purchase of the Apple, which is the item the User has selected.

Calling function buy_item with parameters {'purchase_item_id': 4}


...

User:  maybe the orange


Observation: The user has shown interest in the Orange Juice, which I previously presented as an option along with its price. They have not yet made a definitive choice.
Thoughts: To assist the user in finalizing their selection, I should confirm their choice of Orange Juice and proceed to purchase it, given that it is well within their budget.
Subtask identified: Purchase the Orange Juice for the user, as they have indicated a preference for it.
Calling function buy_item with parameters {'purchase_item_id': 7}



	> AI Assistant: You are about to purchase Orange Juice for 3 dollars. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': 'Purchase of Orange Juice successful. Remaining money after purchase: 997'}

Actions Done: The purchase of Orange Juice was successful, and the remaining money after the purchase is 997.

Thoughts: The user has expressed a preference for the orange juice, which I have already purchased for them. I should inform them of the successful transaction and their updated balance.
Persistent Memory: {}
Summary of Conversation: The user expressed a preference for Orange Juice, which I successfully purchased for them, leaving their remaining balance at 997. I inquired if they needed further assistance.
Shop Assistant: Ah, my dear user, the Orange Juice has been successfully procured for you. Your current balance now stands at 997. Is there anything else I may assist you with today?



User:  exit


In [18]:
shop_agent.shared_variables

{'purchased_items': ['Apple'],
 'money_remaining': 999,
 'item_memory': <agentjo.memory.Memory at 0x157f77c50>,
 'items_searched': [{'Name': 'Orange Juice', 'item_id': 7, 'Cost': 3},
  {'Name': 'Coconut', 'item_id': 8, 'Cost': 10},
  {'Name': 'Apple', 'item_id': 4, 'Cost': 1}],
 'agent': <agentjo.wrapper.ConversationWrapper at 0x157f10860>,
 'Persistent Memory': {},
 'Conversation': ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?',
  'User: anything to drink?',
  'Shop Assistant: Ah, my dear User, I have unearthed a few delightful beverages for your consideration. \nYou may choose from the following: \n- Orange Juice for a mere 3 coins, \n- Coconut at the price of 10 coins, \n- Or an Apple, which can quench your thirst for just 1 coin. \nYou currently possess 1000 coins. How shall we proceed?\n',
  'User: get me that 1 coin deal',
  'Shop Assistant: I have successfully procured the Apple for you, my dear User, for the modest sum of 1 co

# Example Conversation with Rule-based Processing
- Use rule-based methods to give additional prompts / checks as much as possible for math

In [19]:
agent = Agent('Helpful Math Quiz Bot', 
'''You provide a Math question suitable for 6 year old and guides User to solve it.
If User is wrong, provide a creative hint on how to solve the Math Question, but not the answer.
You must ask the next Math question on the third try or when User solves it, whichever is earlier.
Begin with a greeting to the User and ask User the first Math question.
There will be an Oracle to guide you whose inputs are in [[]], do not output anything about the Oracle to the User''',
             llm = llm)

In [20]:
math_conversable_agent = ConversationWrapper(agent, 
                                 persistent_memory = {'Current Math Question': '',
                                                 'Correct Answer for Question': ''})

user_input = ''
cur_tries = 0
while True:
    # Print out current tries
    print('Current Tries:', cur_tries)
    reply = math_conversable_agent.chat(user_input)
    print(math_conversable_agent.agent_name + ':', reply)
    user_input = input('User: ')
    
    # Using LLM to extract out answer to help with processing
    res = strict_json('''Check if there is a number in the message. Extract out a single number from the message if there is. 
If there is no number, output 0.''', 
                      user_input, 
                      output_format = {'Is Number': 'type: bool', 'Extracted Number': 'type: int'},
                      llm = agent.llm,
                      model = agent.kwargs.get('model', 'gpt-3.5-turbo'))
    
    if res['Is Number']:
        user_input = str(res['Extracted Number'])
        ## Do rule-based for solving
        if user_input == str(math_conversable_agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is correct. Ask the next question.]]'
            cur_tries = 0
        elif user_input != str(math_conversable_agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is incorrect. Provide a helpful hint for Correct Answer to Current Math Question without revealing the Correct Answer]]'
            cur_tries += 1
        if cur_tries == 3:
            user_input += '\n[[Oracle: Three tries are up. Give the Correct Answer. Ask the next question.]]'
            cur_tries = 0
    
    print()
    if user_input == 'exit': break

Current Tries: 0


/opt/anaconda3/lib/python3.12/site-packages/strictjson/parse_yaml.py:163: UserWarning: LLM Parsing failed at attempt 1.
Retrying...
  warnings.warn(my_warning, UserWarning)


Thoughts: I need to greet the User and present a math question suitable for a 6-year-old.

Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: The User was greeted and presented with the first math question: What is 2 + 3?

Helpful Math Quiz Bot: Hello! I'm Helpful Math Quiz Bot. Let's have some fun with a math question! What is 2 + 3?



User:  5



Current Tries: 0
Thoughts: The user answered correctly, so I will acknowledge their answer and present the next math question.

Persistent Memory: {'Current Math Question': 'What is 4 + 1?', 'Correct Answer for Question': 5}
Summary of Conversation: The User was greeted and presented with the first math question: What is 2 + 3? The User answered correctly with 5, and the next question is now presented: What is 4 + 1?

Helpful Math Quiz Bot: Great job! That's correct! Now, let's try another one: What is 4 + 1?



...

User:  6



Current Tries: 1
Thoughts: The user answered incorrectly, so I need to provide a hint to help them find the correct answer without giving it away. I will also remind them of the question they are trying to solve.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: The user attempted to answer the first math question about addition but provided an incorrect answer. I have given them a hint to help them solve the question correctly.
Helpful Math Quiz Bot: That’s not quite right! Here’s a hint: Think about how many fingers you would have if you held up 2 fingers on one hand and 3 on the other. What do you get when you put them together?


User:  5



Current Tries: 0
Thoughts: The user has correctly answered the math question. I should congratulate them and present the next math question to keep the quiz going.
Persistent Memory: {'Current Math Question': 'What is 4 + 1?', 'Correct Answer for Question': 5}
Summary of Conversation: The user correctly answered the addition question 2 + 3 after receiving a hint. I congratulated them and provided the next math question, which is 4 + 1.
Helpful Math Quiz Bot: Great job! You got it right! Now, here’s your next question: What is 4 + 1?


User:  exit


# Example Escape Room Conversation
- Using the environment as the 2nd person can mimic a real-world interaction, e.g. robotic environments

### Defining the Escape Room

In [21]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.items = {
            'key': True,
            'lock': True,
            'potion': True
        }
        self.is_door_locked = True
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'unlock door': self.unlock_door,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        description = "You look around and see the following items:\n"
        for item, available in self.items.items():
            if available:
                description += f"- {item}\n"
        if self.is_door_locked:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        if item in self.items and self.items[item]:
            self.inventory.append(item)
            self.items[item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        if 'key' in self.inventory:
            if self.is_door_locked:
                self.is_door_locked = False
                return "You unlocked the door. You can try to exit now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def exit_room(self):
        if not self.is_door_locked:
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "The door is locked. You can't exit yet."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

### Defining the Solver

In [28]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
Output one of the Available Actions using the exact phrasing provided with no additional words.
If previous action fails, try another.''',
llm = llm)

In [29]:
conversable_agent = ConversationWrapper(agent, 
                    persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},
                    person = 'Game')

In [30]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items:
- key
- lock
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- unlock door
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: How to reply
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up the key from the current room.', 'List of Rooms Explored': ''}
Summary of Conversation: The player is in a room with a locked door and several items, including a key, lock, and potion. The next action is to pick up the key to potentially unlock the door.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and see the following items:
- lock
- potion
There is a locked door here.

Your inventory: key

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
-

# Can we solve Harder Rooms
- Live coding to solve Escape Room with multiple rooms

In [31]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.rooms = {
            'Room 1': {
                'items': {
                    'key': True,
                    'lock': False,
                    'potion': True
                },
                'is_door_locked': True
            },
            'Room 2': {
                'items': {
                    'treasure': True
                },
                'is_door_locked': True
            }
        }
        self.current_room = 'Room 1'
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'pick up treasure': lambda: self.pick_up('treasure'),
            'unlock door': self.unlock_door,
            'enter next room': self.enter_next_room,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        room = self.rooms[self.current_room]
        description = f"You look around and see the following items in {self.current_room}:\n"
        for item, available in room['items'].items():
            if available:
                description += f"- {item}\n"
        if room['is_door_locked']:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        room = self.rooms[self.current_room]
        if item in room['items'] and room['items'][item]:
            self.inventory.append(item)
            room['items'][item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        room = self.rooms[self.current_room]
        if 'key' in self.inventory:
            if room['is_door_locked']:
                room['is_door_locked'] = False
                return "You unlocked the door. You can try to enter the next room now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def enter_next_room(self):
        if self.current_room == 'Room 1' and not self.rooms['Room 1']['is_door_locked']:
            self.current_room = 'Room 2'
            return "You enter the next room."
        elif self.current_room == 'Room 2' and not self.rooms['Room 2']['is_door_locked']:
            return self.exit_room()
        else:
            return "The door is locked. You can't enter the next room yet."

    def exit_room(self):
        if self.current_room == 'Room 2':
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "You can't exit from here."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

# Example usage
# escape_room = EscapeRoom()
# print(escape_room.info())
# print(escape_room.play('look around'))
# print(escape_room.play('pick up key'))
# print(escape_room.play('unlock door'))
# print(escape_room.play('enter next room'))
# print(escape_room.play('look around'))
# print(escape_room.play('pick up treasure'))
# print(escape_room.play('exit'))

In [32]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
Output one of the Available Actions using the exact phrasing provided with no additional words.
If previous action fails, try another.''',
llm = llm)

In [33]:
conversable_agent = ConversationWrapper(agent, 
                persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},  
                person = 'Game')

In [34]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items in Room 1:
- key
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- pick up treasure
- unlock door
- enter next room
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: How to reply
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up the key from Room 1.', 'List of Rooms Explored': 'Room 1'}
Summary of Conversation: The player is in Room 1 with a key and a potion, and there is a locked door. The next action is to pick up the key to potentially unlock the door.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and see the following items in Room 1:
- potion
There is a locked door here.

Your inventory: key

Available actions:
- look around
- pick up key
- pick up lock